# Introduction to Data Science 2025

# Week 2

## Exercise 1 | Titanic: data preprocessing and imputation
<span style="font-weight: bold"> *Note: You can find tutorials for NumPy and Pandas under 'Useful tutorials' in the course material.*</span>

Download the [Titanic dataset](https://www.kaggle.com/c/titanic) [train.csv] from Kaggle or <span style="font-weight: 500">directly from the course material</span>, and complete the following exercises. If you choose to download the dataset from Kaggle, you will need to create a Kaggle account unless you already have one, but it is quite straightforward.

The dataset consists of personal information of all the passengers on board the RMS Titanic, along with information about whether they survived the iceberg collision or not.

1. Your first task is to read the data file and print the shape of the data.

    <span style="font-weight: 500"> *Hint 1: You can read them into a Pandas dataframe if you wish.*</span>
    
    <span style="font-weight: 500"> *Hint 2: The shape of the data should be (891, 12).*</span>

In [131]:
import pandas as pd

# assumes the source data as a csv file within the data folder next to the notebook file
df = pd.read_csv("./data/titanic.csv")
print("Shape of data frame:", df.shape)

Shape of data frame: (891, 12)


2. Let's look at the data and get started with some preprocessing. Some of the columns, e.g <span style="font-weight: 500"> *Name*</span>, simply identify a person and are not useful for prediction tasks. Try to identify these columns, and remove them.

    <span style="font-weight: 500"> *Hint: The shape of the data should now be (891, 9).*</span>

In [132]:
df = df.drop(columns=["Name", "Ticket", "Embarked"])
print("Shape of data frame:", df.shape)

Shape of data frame: (891, 9)


3. The column <span style="font-weight: 500">*Cabin*</span> contains a letter and a number. A smart catch at this point would be to notice that the letter stands for the deck level on the ship. Keeping just the deck information would be more informative when developing, e.g. a classifier that predicts whether a passenger survived. The next step in our preprocessing will be to add a new column to the dataset, which consists simply of the deck letter. You can then remove the original <span style="font-weight: 500">*Cabin*</span>-column.

<span style="font-weight: 500">*Hint: The deck letters should be ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T'].*</span>

In [133]:
df = df.assign(Deck=df["Cabin"].str[0])
df = df.drop(columns=["Cabin"])

print("Deck column of first few rows:")
print(df["Deck"].head())

Deck column of first few rows:
0    NaN
1      C
2    NaN
3      C
4    NaN
Name: Deck, dtype: object


4. You’ll notice that some of the columns, such as the previously added deck number, are [categorical](https://en.wikipedia.org/wiki/Categorical_variable). To preprocess the categorical variables so that they're ready for further computation, we need to avoid the current string format of the values. This means the next step for each categorical variable is to transform the string values to numeric ones, that correspond to a unique integer ID representative of each distinct category. This process is called label encoding and you can read more about it [here](https://pandas.pydata.org/docs/user_guide/categorical.html).

    <span style="font-weight: 500">*Hint: Pandas can do this for you.*</span>

In [134]:
df = df.assign(Deck=df["Deck"].astype("category"))
df = df.assign(Deck=df["Deck"].cat.codes)

df = df.assign(Sex=df["Sex"].astype("category"))
df = df.assign(Sex=df["Sex"].cat.codes)

print("Data after converting categorical variables to codes:")
print(df.head())

print(f"\nData types after label encoding:")
print(df.dtypes)

Data after converting categorical variables to codes:
   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Deck
0            1         0       3    1  22.0      1      0   7.2500    -1
1            2         1       1    0  38.0      1      0  71.2833     2
2            3         1       3    0  26.0      0      0   7.9250    -1
3            4         1       1    0  35.0      1      0  53.1000     2
4            5         0       3    1  35.0      0      0   8.0500    -1

Data types after label encoding:
PassengerId      int64
Survived         int64
Pclass           int64
Sex               int8
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Deck              int8
dtype: object


5. Next, let's look into missing value **imputation**. Some of the rows in the data have missing values, e.g when the cabin number of a person is unknown. Most machine learning algorithms have trouble with missing values, and they need to be handled during preprocessing:

    a) For continuous variables, replace the missing values with the mean of the non-missing values of that column.

    b) For categorical variables, replace the missing values with the mode of the column.

    <span style="font-weight: 500">*Remember: Even though in the previous step we transformed categorical variables into their numeric representation, they are still categorical.*</span>

In [135]:
print("Missing values before imputation:")
print(df.isnull().sum())

age_mean = df["Age"].mean()
df["Age"] = df["Age"].fillna(age_mean)

print(f"\nFilled missing Age values with mean: {age_mean:.2f}")

# -1 represents a missing value, exclude from mode
valid_deck_values = df["Deck"][df["Deck"] != -1]

if not valid_deck_values.empty:
    deck_mode = valid_deck_values.mode()[0]
    print(f"Deck mode (excluding missing values): {deck_mode}")

    df["Deck"] = df["Deck"].replace(-1, deck_mode)
else:
    print("No valid deck values found, keeping -1 as missing indicator")

print(f"\nDataset shape after imputation: {df.shape}")
print(df.head())

Missing values before imputation:
PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Deck             0
dtype: int64

Filled missing Age values with mean: 29.70
Deck mode (excluding missing values): 2

Dataset shape after imputation: (891, 9)
   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Deck
0            1         0       3    1  22.0      1      0   7.2500     2
1            2         1       1    0  38.0      1      0  71.2833     2
2            3         1       3    0  26.0      0      0   7.9250     2
3            4         1       1    0  35.0      1      0  53.1000     2
4            5         0       3    1  35.0      0      0   8.0500     2


6. At this point, all data is numeric. Write the data, with the modifications we made, to a  <span style="font-weight: 500"> .csv</span> file. Then, write another file, this time in <span style="font-weight: 500">JSON</span> format, with the following structure:

In [136]:
#[
#    {
#        "Deck": 0,
#        "Age": 20,
#        "Survived", 0
#        ...
#    },
#    {
#        ...
#    }
#]

In [137]:
df.to_csv(path_or_buf="./titanic_preprocessed.csv")
df.to_json(path_or_buf="./titanic_preprocessed.json", orient="records")

Study the records and try to see if there is any evident pattern in terms of chances of survival.

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 2 | Titanic 2.0: exploratory data analysis

In this exercise, we’ll continue to study the Titanic dataset from the last exercise. Now that we have done some preprocessing, it’s time to look at the data with some exploratory data analysis.

1. First investigate each feature variable in turn. For each categorical variable, find out the mode, i.e., the most frequent value. For numerical variables, calculate the median value.

In [147]:
print("Dataframe after preprocessing:")
print(df.head())

print(f"\nMode of categorical variables:")
print(f"Survived: {df["Survived"].mode()[0]}")
print(f"Pclass: {df["Pclass"].mode()[0]}")
print(f"Sex: {df["Sex"].mode()[0]}")
print(f"Deck: {df["Deck"].mode()[0]}")

print(f"\nMedian of numerical variables:")
print(f"Age: {df["Age"].median():.2f}")
print(f"SibSp: {df["SibSp"].median():.2f}")
print(f"Parch: {df["Parch"].median():.2f}")
print(f"Fare: {df["Fare"].median():.2f}")


Dataframe after preprocessing:
   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Deck
0            1         0       3    1  22.0      1      0   7.2500     2
1            2         1       1    0  38.0      1      0  71.2833     2
2            3         1       3    0  26.0      0      0   7.9250     2
3            4         1       1    0  35.0      1      0  53.1000     2
4            5         0       3    1  35.0      0      0   8.0500     2

Mode of categorical variables:
Survived: 0
Pclass: 3
Sex: 1
Deck: 2

Median of numerical variables:
Age: 29.70
SibSp: 0.00
Parch: 0.00
Fare: 14.45


2. Next, combine the modes of the categorical variables, and the medians of the numerical variables, to construct an imaginary “average survivor”. This "average survivor" should represent the typical passenger of the class of passengers who survived. Also following the same principle, construct the “average non-survivor”.

    <span style="font-weight: 500">*Hint 1: What are the average/most frequent variable values for a non-survivor?*</span>
    
    <span style="font-weight: 500">*Hint 2: You can split the dataframe in two: one subset containing all the survivors and one consisting of all the non-survivor instances. Then, you can use the summary statistics of each of these dataframe to create a prototype "average survivor" and "average non-survivor", respectively.*</span>

In [149]:
survivors = df[df["Survived"] == 1]
non_survivors = df[df["Survived"] == 0]

def print_average(df):
    print(f"Pclass: {df["Pclass"].mode()[0]}")
    print(f"Sex: {df["Sex"].mode()[0]}")
    print(f"Deck: {df["Deck"].mode()[0]}")
    print(f"Age: {df["Age"].median():.2f}")
    print(f"SibSp: {df["SibSp"].median():.2f}")
    print(f"Parch: {df["Parch"].median():.2f}")
    print(f"Fare: {df["Fare"].median():.2f}")

print(f"Average survivor:")
print_average(survivors)

print(f"\nAverage non-survivor:")
print_average(non_survivors)

Average survivor:
Pclass: 1
Sex: 0
Deck: 2
Age: 29.70
SibSp: 0.00
Parch: 0.00
Fare: 26.00

Average non-survivor:
Pclass: 3
Sex: 1
Deck: 2
Age: 29.70
SibSp: 0.00
Parch: 0.00
Fare: 10.50


3. Next, let's study the distributions of the variables in the two groups (survivor/non-survivor). How well do the average cases represent the respective groups? Can you find actual passengers that are very similar to the (average) representative of their own group? Can you find passengers that are very similar to the (average) representative of the other group?

    <span style="font-weight: 500">*Note: Feel free to choose EDA methods according to your preference: non-graphical/graphical, static/interactive - anything goes.*</span>

In [140]:
# Use this cell for your code

4. Next, let's continue the analysis by looking into pairwise and multivariate relationships between the variables in the two groups. Try to visualize two variables at a time using, e.g., scatter plots and use a different color to encode the survival status.

    <span style="font-weight: 500">*Hint 1: You can also check out Seaborn's pairplot function, if you wish.*</span>

    <span style="font-weight: 500">*Hint 2: To better show many data points with the same value for a given variable, you can use either transparency or ‘jitter’.*</span>

In [141]:
# Use this cell for your code

5. Finally, recall the preprocessing we did in the first exercise. What can you say about the effect of the choices that were made to use the mode and mean to impute missing values, instead of, for example, ignoring passengers with missing data?

*Use this (markdown) cell for your written answer*

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 3 | Working with text data 2.0

This exercise is related to the second exercise from last week. Find the saved <span style="font-weight: 500">pos.txt</span> and <span style="font-weight: 500">neg.txt</span> files, or, alternatively, you can find the week 1 example solutions on the MOOC platform after Tuesday.

1. Find the most common words in each file (positive and negative). Examine the results. Do they tend to be general terms relating to the nature of the data? How well do they indicate positive/negative sentiment?

In [142]:
# Use this cell for your code

2. Compute a [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) vector for each of the two text files, and make them into a <span style="font-weight: 500">2 x m</span> matrix, where <span style="font-weight: 500">m</span> is the number of unique words in the data. The problem with using the most common words in a review to analyze its contents is that words that are common overall will be common in all reviews (both positive and negative). This means that they probably are not good indicators about the sentiment of a specific review. TF/IDF stands for Term Frequency / Inverse Document Frequency (here the reviews are the documents), and is designed to help by taking into consideration not just the number of times a term occurs (term frequency), but also how many times a word exists in other reviews as well (inverse document frequency). You can use any variant of the formula, as well as off-the-shelf implementations. <span style="font-weight: 500">*Hint: You can use [sklearn](http://scikit-learn.org/).*</span>

In [143]:
# Use this cell for your code

3. List the words with the highest TF/IDF score in each class (positive | negative), and compare them to the most common words. What do you notice? Did TF/IDF work as expected?

In [144]:
# Use this cell for your code

4. Plot the words in each class with their corresponding TF/IDF scores. Note that there will be a lot of words, so you’ll have to think carefully to make your chart clear! If you can’t plot them all, plot a subset – think about how you should choose this subset.

    <span style="font-weight: 500">*Hint: you can use word clouds. But feel free to challenge yourselves to think of any other meaningful way to visualize this information!*</span>

In [145]:
# Use this cell for your code

**Remember to submit your code on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

## Exercise 4 | Junk charts

There’s a thriving community of chart enthusiasts who keep looking for statistical graphics that they find inappropriate, and which they call “junk charts”, and who often also propose ways to improve them.

1. Find at least three statistical visualizations you think are not very good and identify their problems. Copying examples from various junk chart websites is not accepted – you should find your own junk charts, out in the wild. You should be able to find good (or rather, bad) examples quite easily since a significant fraction of charts can have at least *some* issues. The examples you choose should also have different problems, e.g., try to avoid collecting three bar charts, all with problematic axes. Instead, try to find as interesting and diverse examples as you can.

2. Try to produce improved versions of the charts you selected. The data is of course often not available, but perhaps you can try to extract it, at least approximately, from the chart. Or perhaps you can simulate data that looks similar enough to make the point.



**Submit a PDF with all the charts (the ones you found and the ones you produced).**